In [1]:
import pickle
import pandas as pd
import numpy as np
df = pd.read_csv('Renewable.csv')
df['Time'] = pd.to_datetime(df['Time'])
df = df[df['Energy delta[Wh]'] != 0]
df1 = df[['Time','Energy delta[Wh]']].copy('Deep')
df1 = df1.set_index('Time')

years = [2017, 2018, 2019, 2020, 2021, 2022]
df2 = df1.reset_index('Time')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler_data = scaler.fit_transform(df1)
#scaler_data = scaler_data[:, 0]  # Assuming filtered_dfx has only one column


print("NaNs after scaling:", np.isnan(scaler_data).any())


index = np.arange(0, len(scaler_data), 1)
scaler_df = pd.DataFrame(scaler_data, index=index, columns=['Energy delta[Wh]'])


train_size = int(len(scaler_df) * 0.8)
train, test = scaler_df.iloc[0:train_size], scaler_df.iloc[train_size:len(scaler_df)]

print("Train size:", len(train), "Test size:", len(test))
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Define the time steps
n_steps = 24

X_train, y_train = create_dataset(train, train['Energy delta[Wh]'], n_steps)

X_test, y_test = create_dataset(test, test['Energy delta[Wh]'], n_steps)

print("Training data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.models import Sequential
lstm_model = keras.Sequential([
    LSTM(units=128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dropout(0.2),
    Dense(units=1)
])

lstm_model.compile(
    loss='mean_squared_error',
    optimizer=keras.optimizers.Adam(0.001)
)

# Callbacks (Corrected)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) # patience=10
model_checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True)
history = lstm_model.fit(
    X_train, y_train,
    epochs=250,
    batch_size=24,
    validation_split=0.1,
    verbose=1,
    shuffle=False,
    callbacks=[early_stopping, model_checkpoint]
)

lstm_pred = lstm_model.predict(X_test)
df_lstm_final = test[lstm_pred.shape[0]*-1:].copy()
df_lstm_final['Prediction'] = lstm_pred[:,0]
df_reset_index = df1.reset_index()
df_lstm_final['Date'] = df_reset_index['Time']
df_lstm_final = df_lstm_final.set_index(["Date"], drop=True)

#df_lstm_final.head()
df_lstm_final.to_csv('lstm_final.csv')
train2 = train.copy()
train2.index = df2.index[:len(train)]
train2['Energy delta[Wh]'] = scaler.inverse_transform(train2['Energy delta[Wh]'].to_numpy().reshape(-1, 1))
train2['Time'] = df2['Time']
train2 = train2.set_index(['Time'])
train2.to_csv('train2.csv')
lstm_model.save('lstm_model.h5')  

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


NaNs after scaling: False
Train size: 76740 Test size: 19186
Training data shape: (76716, 24, 1) (76716,)
Test data shape: (19162, 24, 1) (19162,)
Epoch 1/250


C:\Users\braha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2877/2877 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - loss: 0.0110 - val_loss: 0.0087
Epoch 2/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 40s 14ms/step - loss: 0.0074 - val_loss: 0.0087
Epoch 3/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.0072 - val_loss: 0.0088
Epoch 4/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - loss: 0.0070 - val_loss: 0.0090
599/599 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step


In [ ]:
import pickle
import pandas as pd
import numpy as np
df = pd.read_csv('Renewable.csv')
df['Time'] = pd.to_datetime(df['Time'])
df = df[df['Energy delta[Wh]'] != 0]
df1 = df[['Time','Energy delta[Wh]']].copy('Deep')
df1 = df1.set_index('Time')

years = [2017, 2018, 2019, 2020, 2021, 2022]
df2 = df1.reset_index('Time')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler_data = scaler.fit_transform(df1)
#scaler_data = scaler_data[:, 0]  # Assuming filtered_dfx has only one column


print("NaNs after scaling:", np.isnan(scaler_data).any())


index = np.arange(0, len(scaler_data), 1)
scaler_df = pd.DataFrame(scaler_data, index=index, columns=['Energy delta[Wh]'])


train_size = int(len(scaler_df) * 0.8)
train, test = scaler_df.iloc[0:train_size], scaler_df.iloc[train_size:len(scaler_df)]

print("Train size:", len(train), "Test size:", len(test))
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Define the time steps
n_steps = 24

X_train, y_train = create_dataset(train, train['Energy delta[Wh]'], n_steps)

X_test, y_test = create_dataset(test, test['Energy delta[Wh]'], n_steps)

print("Training data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.models import Sequential
lstm_model = keras.Sequential([
    LSTM(units=128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dropout(0.2),
    Dense(units=1)
])

lstm_model.compile(
    loss='mean_squared_error',
    optimizer=keras.optimizers.Adam(0.001)
)

# Callbacks (Corrected)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) # patience=10
model_checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True)
history = lstm_model.fit(
    X_train, y_train,
    epochs=250,
    batch_size=24,
    validation_split=0.1,
    verbose=1,
    shuffle=False,
    callbacks=[early_stopping, model_checkpoint]
)

lstm_pred = lstm_model.predict(X_test)
df_lstm_final = test[lstm_pred.shape[0]*-1:].copy()
df_lstm_final['Prediction'] = lstm_pred[:,0]
df_reset_index = df1.reset_index()
df_lstm_final['Date'] = df_reset_index['Time']
df_lstm_final = df_lstm_final.set_index(["Date"], drop=True)

#df_lstm_final.head()
df_lstm_final.to_csv('lstm_final.csv')
train2 = train.copy()
train2.index = df2.index[:len(train)]
train2['Energy delta[Wh]'] = scaler.inverse_transform(train2['Energy delta[Wh]'].to_numpy().reshape(-1, 1))
train2['Time'] = df2['Time']
train2 = train2.set_index(['Time'])
train2.to_csv('train2.csv')
lstm_model.save('lstm_model.keras')

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

NaNs after scaling: False
Train size: 76740 Test size: 19186
Training data shape: (76716, 24, 1) (76716,)
Test data shape: (19162, 24, 1) (19162,)
Epoch 1/250


C:\Users\braha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2877/2877 ━━━━━━━━━━━━━━━━━━━━ 39s 13ms/step - loss: 0.0110 - val_loss: 0.0087
Epoch 2/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.0074 - val_loss: 0.0087
Epoch 3/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.0072 - val_loss: 0.0087
Epoch 4/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.0070 - val_loss: 0.0086
Epoch 5/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - loss: 0.0069 - val_loss: 0.0085
Epoch 6/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - loss: 0.0068 - val_loss: 0.0085
Epoch 7/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 45s 16ms/step - loss: 0.0068 - val_loss: 0.0084
Epoch 8/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 45s 16ms/step - loss: 0.0067 - val_loss: 0.0084
Epoch 9/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 45s 16ms/step - loss: 0.0067 - val_loss: 0.0084
Epoch 10/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 46s 16ms/step - loss: 0.0066 - val_loss: 0.0084
Epoch 11/250
2877/2877 ━━━━━━━━━━━━━━━━━━━━ 46s 16ms/step - loss: 0.0066 - val_loss: 0.0083
Epoch 12/